In [1]:
!pip install gspread
!pip install oauth2client
!pip install gspread_dataframe


In [2]:
# Import the necessary libraries
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default
from gspread_dataframe import set_with_dataframe

# Authenticate the user
auth.authenticate_user()
creds, _ = default()

# Open the spreadsheet by the URL
gsheets = gspread.authorize(creds)
planilha = gsheets.open_by_url('https://docs.google.com/spreadsheets/d/11-IQ3x8gJIg-M6SUprNuQtfdxTLbNO0S5IYfFdtt3I4/edit?usp=sharing')

# Read the data from the spreadsheet and put them in a dataframe
data = planilha.worksheet('engenharia_de_software').get_all_values()
df = pd.DataFrame(data[3:], columns=data[2:])


In [3]:
# Define a function to calculate the mean of the tests
def calculate_mean(p1, p2, p3):
  try:
    # Convert the grades to numbers
    val1 = float(p1)
    val2 = float(p2)
    val3 = float(p3)
    # Calculate the arithmetic mean
    mean = (val1 + val2 + val3) / 3
    return mean
  except Exception as e:
    return 0.00


# Define a function to calculate the student's situation
def calcular_situacao(mean, faltas):
  # Set the total number of classes
  total_aulas = 60
  try:
    # Convert absences to numbers
    valfaltas = float(faltas)
    # Calculate the percentage of absences
    porcentagem_faltas = (valfaltas / total_aulas) * 100
    # Check whether the student has failed due to absence
    if porcentagem_faltas > 25:
      situacao = "Reprovado por Falta"
    # Otherwise, check the student's average
    else:
      # If the average is less than 5, the student is failing by grade
      if mean < 50:
        situacao = "Reprovado por Nota"
      # If the average is greater than or equal to 7, the student is approved
      elif mean >= 50 and mean < 70:
        # If the average is between 5 and 7, the student is in the final exam
        situacao = "Exame Final"
      elif mean >= 70:
        situacao = "Aprovado"
    # Return the situation
    return situacao
  except Exception as e:
    return "Não Calculado"

# Define a function to calculate the grade for final approval
def calcular_naf(mean, situacao):
  try:
    # Check if the student is in the final exam
    if situacao == "Exame Final":
      # Apply the formula to calculate the grade for final approval
      return float(70) - float(mean)
    # Otherwise, fill the field with 0
    else:
      return 0
  except Exception as e:
    return mean

# Apply the function to calculate the student's situation and create a column called 'Situacao'
df['Situação'] = df.apply(lambda row: calcular_situacao(calculate_mean(row['P1'], row['P2'], row['P3']), row['Faltas']), axis=1)
# Apply the function to calculate the grade for final approval and create a column called 'NAF'
df['Nota para Aprovação Final'] = df.apply(lambda row: calcular_naf(calculate_mean(row['P1'], row['P2'], row['P3']), calcular_situacao(calculate_mean(row['P1'], row['P2'], row['P3']), row['Faltas'])), axis=1)

worksheet = planilha.worksheet('engenharia_de_software')

# Specify the starting row and column where I want to write the data
start_line = 4  # replace with the desired line number
start_column = 1  # replace with the desired column number

# Write data back to the worksheet from the specified row and column
for i, row in enumerate(df.itertuples(), start=start_line):
    for j, value in enumerate(row[start_column:], start=start_column):
        worksheet.update_cell(i, j, str(value))
